# 如何合并多个检索器的结果
[EnsembleRetriever](https://python.langchain.com/api_reference/langchain/retrievers/langchain.retrievers.ensemble.EnsembleRetriever.html) 支持对多个[检索器](/docs/concepts/retrievers/)的结果进行集成。它通过一组[BaseRetriever](https://python.langchain.com/api_reference/core/retrievers/langchain_core.retrievers.BaseRetriever.html)对象进行初始化。EnsembleRetriever会基于[倒数排序融合算法](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf)对成员检索器的结果进行重新排序。
通过结合不同算法的优势，`EnsembleRetriever`能够实现比任何单一算法更优的性能。
最常见的模式是将稀疏检索器（如BM25）与密集检索器（如嵌入相似度）结合使用，因为它们的优势是互补的。这也被称为“混合搜索”。稀疏检索器擅长基于关键词查找相关文档，而密集检索器则擅长基于语义相似度查找相关文档。
## 基本用法
下面我们将演示如何将[BM25Retriever](https://python.langchain.com/api_reference/community/retrievers/langchain_community.retrievers.bm25.BM25Retriever.html)与源自[FAISS向量数据库](https://python.langchain.com/api_reference/community/vectorstores/langchain_community.vectorstores.faiss.FAISS.html)的检索器进行集成。

In [9]:
%pip install --upgrade --quiet  rank_bm25 > /dev/null

In [3]:
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

doc_list_1 = [
    "I like apples",
    "I like oranges",
    "Apples and oranges are fruits",
]

# initialize the bm25 retriever and faiss retriever
bm25_retriever = BM25Retriever.from_texts(
    doc_list_1, metadatas=[{"source": 1}] * len(doc_list_1)
)
bm25_retriever.k = 2

doc_list_2 = [
    "You like apples",
    "You like oranges",
]

embedding = OpenAIEmbeddings()
faiss_vectorstore = FAISS.from_texts(
    doc_list_2, embedding, metadatas=[{"source": 2}] * len(doc_list_2)
)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 2})

# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)

In [4]:
docs = ensemble_retriever.invoke("apples")
docs

[Document(page_content='I like apples', metadata={'source': 1}),
 Document(page_content='You like apples', metadata={'source': 2}),
 Document(page_content='Apples and oranges are fruits', metadata={'source': 1}),
 Document(page_content='You like oranges', metadata={'source': 2})]

## 运行时配置
我们还可以在运行时使用[可配置字段](/docs/how_to/configure)来单独配置检索器。下面我们专门为FAISS检索器更新"top-k"参数：

In [5]:
from langchain_core.runnables import ConfigurableField

faiss_retriever = faiss_vectorstore.as_retriever(
    search_kwargs={"k": 2}
).configurable_fields(
    search_kwargs=ConfigurableField(
        id="search_kwargs_faiss",
        name="Search Kwargs",
        description="The search kwargs to use",
    )
)

ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)

In [6]:
config = {"configurable": {"search_kwargs_faiss": {"k": 1}}}
docs = ensemble_retriever.invoke("apples", config=config)
docs

[Document(page_content='I like apples', metadata={'source': 1}),
 Document(page_content='You like apples', metadata={'source': 2}),
 Document(page_content='Apples and oranges are fruits', metadata={'source': 1})]

请注意，这仅从FAISS检索器返回一个来源，因为我们在运行时传入了相关配置。